In [10]:
import os
import time
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

load_dotenv()

True

In [11]:
import logging
from logging.handlers import RotatingFileHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

file_formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s")
log_dir = os.path.join(os.getcwd(), "logs")
file_handler = RotatingFileHandler(
    filename=os.path.join(log_dir, "scraper.log"),
    maxBytes=10*1024*1024,
    backupCount=5
)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(file_formatter)

logger.addHandler(file_handler)

In [14]:
def check_for_new_offers(location, url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        search_result = soup.find("h2", class_="SearchResults-desktop")

        if search_result:
            message = search_result.get_text(strip=True)

            if "Aucun logement" not in message:
                log_content(location, message)
                send_email(location, message, url, os.getenv("RECEIVER_EMAIL"))


def log_content(location, message):
    log_entry = f"{location.ljust(60)}: {message}\n"
    logger.info(log_entry)


def send_email(location, message, url, receiver_email):
    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")

    subject = "New Apartment Offers"
    body = f"There are new apartment offers in \
        {location}: {message}\nurl : {url}"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            text = msg.as_string()
            server.sendmail(sender_email, receiver_email, text)
            print(f"Email notification sent for {location}")
    except Exception as e:
        print(f"Error sending email: {str(e)}")

In [15]:
url_1 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.6881371_48.6461896_7.8360646_48.491861"
url_2 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.7553056_48.5875003_7.7563369_48.5868378"
url_3 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.681771588321607_48.628345309716735_7.817717211678392_48.53841329028326"
url_4 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.7368076_48.5863384_7.7378865_48.5856638"
url_5 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.7011138_48.635168_7.803662_48.5981431"
url_6 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.6605309_48.5695747_7.7068096_48.5389959"
url_7 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.6659373_48.6211944_7.708777_48.5972107"
url_8 = "https://trouverunlogement.lescrous.fr/tools/32/search?bounds=7.6887965_48.6323965_7.7216977_48.6143763"

locations = {
    "Strasbourg": url_1,
    "Bibliothque_nationale_universitaire_de_strasbourg": url_2,
    "Epitech_strasbourg": url_3,
    "Ecole_nationale_superieure_d_architecture_de_strasbourg": url_4,
    "Schiltigheim": url_5,
    "Lingolsheim": url_6,
    "Oberhausbergen": url_7,
    "Niederhausbergen": url_8,
}

In [17]:
logger.info("Starting the scraper")
while True:
    for location, url in locations.items():
        check_for_new_offers(location, url)

    time.sleep(60)

KeyboardInterrupt: 